In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pd.options.display.max_rows = 999
pd.options.display.max_columns=999

In [ ]:
train = pd.read_json('/kaggle/input/two-sigma-connect-rental-listing-inquiries/train.json.zip')
test = pd.read_json('/kaggle/input/two-sigma-connect-rental-listing-inquiries/test.json.zip')
all_data = pd.concat([train,test])
all_data.head(5)

In [ ]:
# all_data['manager_id'].value_counts()

In [ ]:
#1) all_data = pd.concat([train,test]).reset_index(drop=True)# reset_index
#2) 
all_data = pd.concat([train,test],ignore_index=True)

# all_data['created'] = all_data['created'].astype('datetime64')
# all_data['created_year'] = all_data['created'].dt.year
# all_data['created_month'] = all_data['created'].dt.month
# all_data['created_day'] = all_data['created'].dt.day 
# all_data['created_hour'] = all_data['created'].dt.hour
# #all_data['created_week'] = all_data['created'].dt.week

all_data['num_photos'] = all_data['photos'].apply(len)
all_data['num_features'] = all_data['features'].apply(len)
all_data

In [ ]:
#description
# text처리 
# 1) machine-learning:count vectorizer(단어 발생 빈도->중요성 파악), tf-idf(단어 발생 빈도->중요성 파악+가중치) *가중치:1~0. 불용어의 가중치 낮춤.
# 2) deep-learning * 임베딩 개념

# from sklearn.feature_extraction.text import TfidfVectorizer

# tf = TfidfVectorizer()
# description_text = tf.fit_transform(all_data['description'])

# description_text #Sparse matrix: 대용량 데이터 처리에 단어의 종류 55859

In [ ]:
# description_text[0] #Sparse matrix: 대용량 데이터 처리에 단어의 종류 55859
# print(description_text[0]) # 총 52개의 데이터 가지고 있음 (<1x55859 sparse matrix of type '<class 'numpy.float64'>' with 52 stored elements in Compressed Sparse Row format>)
#  (0, 54290)	0.03894011143583986
# 54290: 많이 등장할 수록 숫자가 작아짐
# 0.03894011143583986: 가중치

In [ ]:
# 차원 축소
# pca, svd
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10) #n_components를 변경해가면서 적정한 차원을 설정 (for loop)
description_svd = svd.fit_transform(description_text)
description_svd

In [ ]:
# all_data['description1'] = description_svd[:,0]
# all_data['description2'] = description_svd[:,1]
all_data = pd.concat([all_data,pd.DataFrame(description_svd)],axis=1)


In [ ]:
all_data

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data['building_id'] = le.fit_transform(all_data['building_id'])
all_data['display_address'] = le.fit_transform(all_data['display_address'])
all_data['street_address'] = le.fit_transform(all_data['street_address'])
all_data['manager_id'] = le.fit_transform(all_data['manager_id'])

all_data

In [ ]:
all_data2 = all_data[all_data.columns[all_data.dtypes!='object']]
all_data2

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

display(train2, test2)

In [ ]:
from catboost import CatBoostClassifier
cbc = CatBoostClassifier() # 모델 돌릴 때만 GPU를 통해 속도 개선 가능. 전처리시에는 multi-process를 통해서 속도를 개선해야 한다.
cbc.fit(train2,train['interest_level'])
result = cbc.predict_proba(test2) # predict()

result # 기본 모델에서는 알파벳 순서로 컬럼을 정렬

In [ ]:
sub = pd.read_csv('/kaggle/input/two-sigma-connect-rental-listing-inquiries/sample_submission.csv.zip')
sub.iloc[:,1:] = result
sub.columns = ['listing_id','high','low','medium']
sub
sub.to_csv('sub.csv', index=False)